# Setup

In [ ]:
!nvidia-smi

Sat Nov 16 14:59:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!apt-get update

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,110 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,482 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [ ]:
!apt-get install -y cuda

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 nvidia-driver-565-open : Depends: libnvidia-gl-565 (= 565.57.01-0ubuntu1) but it is not installable
                          Depends: nvidia-dkms-565-open (= 565.57.01-0ubuntu1) but it is not installable
                          Depends: nvidia-kernel-common-565 (= 565.57.01-0ubuntu1) but it is not installable
                          Depends: nvidia-kernel-source-565-open (= 565.57.01-0ubuntu1) but it is not installable
                          Depends: libnvidia-compute-565 (= 565.57.01-0ubuntu1) but it is not installable
                          D

# 1. Implémentation Baseline

In [ ]:
%%writefile matrice_multiplication_global_baseline.cu

Writing matrice_multiplication_global_baseline.cu


In [ ]:
%%writefile matrice_multiplication_global_baseline.cu
#include <cuda_runtime.h>
#include <iostream>

#define N 512  // Taille des matrices carrées (N x N)

__global__ void matrixMultiply(const float* A, const float* B, float* C, int n) {
    // Obtenir l'indice de la ligne et de la colonne pour le thread actuel
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Calculer l'élément de la matrice résultat C
    if (row < n && col < n) {
        float sum = 0.0f;
        for (int k = 0; k < n; ++k) {
            sum += A[row * n + k] * B[k * n + col];
        }
        C[row * n + col] = sum;
    }
}

int main() {
    int size = N * N * sizeof(float);
    float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    // Allocation de la mémoire hôte
    h_A = (float*)malloc(size);
    h_B = (float*)malloc(size);
    h_C = (float*)malloc(size);

    // Initialisation des matrices A et B avec des valeurs arbitraires
    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f;  // exemple de valeurs
        h_B[i] = 1.0f;
    }

    // Allocation de la mémoire sur le périphérique (GPU)
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copie des matrices A et B de l'hôte vers le périphérique
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Configuration de la grille et des blocs
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Lancer le noyau CUDA
    matrixMultiply<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Copier le résultat du périphérique vers l'hôte
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Vérification du résultat (optionnel)
    std::cout << "Exemple de sortie (C[0][0]): " << h_C[0] << std::endl;

    // Libération de la mémoire
    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Overwriting matrice_multiplication_global_baseline.cu


In [ ]:
!nvcc matrice_multiplication_global_baseline.cu -o matrice_multiplication_global_baseline

In [ ]:
!./matrice_multiplication_global_baseline

Exemple de sortie (C[0][0]): 512


In [ ]:
!nvprof ./matrice_multiplication_global_baseline

==964== NVPROF is profiling process 964, command: ./matrice_multiplication_global_baseline
Exemple de sortie (C[0][0]): 512
==964== Profiling application: ./matrice_multiplication_global_baseline
==964== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   81.57%  1.1357ms         1  1.1357ms  1.1357ms  1.1357ms  matrixMultiply(float const *, float const *, float*, int)
                   12.56%  174.88us         2  87.438us  87.390us  87.486us  [CUDA memcpy HtoD]
                    5.87%  81.694us         1  81.694us  81.694us  81.694us  [CUDA memcpy DtoH]
      API calls:   95.71%  68.915ms         3  22.972ms  3.3660us  68.840ms  cudaMalloc
                    3.48%  2.5036ms         3  834.54us  233.16us  1.9630ms  cudaMemcpy
                    0.31%  223.13us         3  74.375us  12.598us  118.56us  cudaFree
                    0.28%  201.05us         1  201.05us  201.05us  201.05us  cudaLaunchKernel
             

# 2. Optimisation de l'accès à la mémoire globale

In [ ]:
%%writefile matrice_multiplication_global_optimized.cu

Writing matrice_multiplication_global_optimized.cu


In [ ]:
%%writefile matrice_multiplication_global_optimized.cu
#include <cuda_runtime.h>
#include <iostream>

#define N 512  // Taille des matrices carrées (N x N)

// Noyau CUDA pour la multiplication de matrices
__global__ void matrixMultiply(const float* A, const float* B_T, float* C, int n) {
    // Obtenir l'indice de la ligne et de la colonne pour le thread actuel
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Calculer l'élément de la matrice résultat C
    if (row < n && col < n) {
        float sum = 0.0f;
        for (int k = 0; k < n; ++k) {
            sum += A[row * n + k] * B_T[col * n + k]; // B_T est transposée
        }
        C[row * n + col] = sum;
    }
}

// Fonction pour transposer une matrice sur le CPU
void transposeMatrix(const float* src, float* dest, int n) {
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            dest[j * n + i] = src[i * n + j];
        }
    }
}

int main() {
    int size = N * N * sizeof(float);
    float *h_A, *h_B, *h_B_T, *h_C;
    float *d_A, *d_B_T, *d_C;

    // Allocation de la mémoire hôte
    h_A = (float*)malloc(size);
    h_B = (float*)malloc(size);
    h_B_T = (float*)malloc(size); // Pour stocker la matrice transposée
    h_C = (float*)malloc(size);

    // Initialisation des matrices A et B avec des valeurs arbitraires
    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f;  // Exemple de valeurs
        h_B[i] = 1.0f;
    }

    // Transposer la matrice B sur le CPU
    transposeMatrix(h_B, h_B_T, N);

    // Allocation de la mémoire sur le périphérique (GPU)
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B_T, size);
    cudaMalloc((void**)&d_C, size);

    // Copie des matrices A et B_T (transposée) de l'hôte vers le périphérique
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B_T, h_B_T, size, cudaMemcpyHostToDevice);

    // Configuration de la grille et des blocs
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Lancer le noyau CUDA
    matrixMultiply<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B_T, d_C, N);

    // Copier le résultat du périphérique vers l'hôte
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Vérification du résultat (optionnel)
    std::cout << "Exemple de sortie (C[0][0]): " << h_C[0] << std::endl;

    // Libération de la mémoire
    free(h_A);
    free(h_B);
    free(h_B_T);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B_T);
    cudaFree(d_C);

    return 0;
}



Overwriting matrice_multiplication_global_optimized.cu


In [ ]:
!nvcc matrice_multiplication_global_optimized.cu -o matrice_multiplication_global_optimized

In [ ]:
!./matrice_multiplication_global_optimized

Exemple de sortie (C[0][0]): 512


In [ ]:
!nvprof ./matrice_multiplication_global_optimized

==1185== NVPROF is profiling process 1185, command: ./matrice_multiplication_global_optimized
Exemple de sortie (C[0][0]): 512
==1185== Profiling application: ./matrice_multiplication_global_optimized
==1185== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   95.92%  6.5611ms         1  6.5611ms  6.5611ms  6.5611ms  matrixMultiply(float const *, float const *, float*, int)
                    2.56%  175.39us         2  87.694us  87.614us  87.774us  [CUDA memcpy HtoD]
                    1.52%  104.00us         1  104.00us  104.00us  104.00us  [CUDA memcpy DtoH]
      API calls:   91.55%  92.703ms         3  30.901ms  7.0670us  92.624ms  cudaMalloc
                    7.85%  7.9501ms         3  2.6500ms  237.32us  7.4050ms  cudaMemcpy
                    0.23%  232.00us         3  77.334us  14.623us  119.76us  cudaFree
                    0.20%  202.60us         1  202.60us  202.60us  202.60us  cudaLaunchKernel
       

# 3. Utilisation de la mémoire partagée

In [ ]:
%%writefile matrice_multiplication_shared.cu

Writing matrice_multiplication_shared.cu


In [ ]:
%%writefile matrice_multiplication_shared.cu
#include <cuda_runtime.h>
#include <iostream>

#define TILE_SIZE 16  // Taille de la tuile (TILE_SIZE x TILE_SIZE)
#define N 512         // Taille des matrices carrées (N x N)

// Noyau CUDA pour la multiplication matricielle optimisée avec mémoire partagée
__global__ void matrixMultiplyShared(const float* A, const float* B, float* C, int n) {
    // Déclaration de mémoire partagée pour stocker les tuiles de A et B
    __shared__ float tile_A[TILE_SIZE][TILE_SIZE];
    __shared__ float tile_B[TILE_SIZE][TILE_SIZE];

    // Calculer les indices de ligne et de colonne pour le thread actuel
    int row = blockIdx.y * TILE_SIZE + threadIdx.y; // Ligne gérée par ce thread
    int col = blockIdx.x * TILE_SIZE + threadIdx.x; // Colonne gérée par ce thread
    float sum = 0.0f; // Accumulateur pour le produit-somme

    // Boucle sur toutes les tuiles nécessaires
    for (int tileIdx = 0; tileIdx < (n + TILE_SIZE - 1) / TILE_SIZE; ++tileIdx) {
        // Charger une tuile de A dans la mémoire partagée
        if (row < n && (tileIdx * TILE_SIZE + threadIdx.x) < n)
            tile_A[threadIdx.y][threadIdx.x] = A[row * n + tileIdx * TILE_SIZE + threadIdx.x];
        else
            tile_A[threadIdx.y][threadIdx.x] = 0.0f;

        // Charger une tuile de B dans la mémoire partagée
        if (col < n && (tileIdx * TILE_SIZE + threadIdx.y) < n)
            tile_B[threadIdx.y][threadIdx.x] = B[(tileIdx * TILE_SIZE + threadIdx.y) * n + col];
        else
            tile_B[threadIdx.y][threadIdx.x] = 0.0f;

        // Synchroniser les threads pour s'assurer que la tuile est complètement chargée
        __syncthreads();

        // Calcul produit-somme sur la tuile actuelle
        for (int k = 0; k < TILE_SIZE; ++k) {
            sum += tile_A[threadIdx.y][k] * tile_B[k][threadIdx.x];
        }

        // Synchroniser les threads avant de charger la prochaine tuile
        __syncthreads();
    }

    // Stocker le résultat final dans la mémoire globale
    if (row < n && col < n) {
        C[row * n + col] = sum;
    }
}

int main() {
    int size = N * N * sizeof(float); // Taille totale en octets des matrices
    float *h_A, *h_B, *h_C;          // Pointeurs pour la mémoire hôte
    float *d_A, *d_B, *d_C;          // Pointeurs pour la mémoire périphérique (GPU)

    // Allocation de mémoire sur l'hôte
    h_A = (float*)malloc(size);
    h_B = (float*)malloc(size);
    h_C = (float*)malloc(size);

    // Initialisation des matrices A et B
    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f; // Chaque élément de A est 1.0
        h_B[i] = 1.0f; // Chaque élément de B est 1.0
    }

    // Allocation de mémoire sur le GPU
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copier les matrices A et B depuis l'hôte vers le GPU
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Configuration des dimensions des threads et des blocs
    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE); // Chaque bloc contient TILE_SIZE x TILE_SIZE threads
    dim3 blocksPerGrid((N + TILE_SIZE - 1) / TILE_SIZE,
                       (N + TILE_SIZE - 1) / TILE_SIZE); // Nombre de blocs nécessaires

    // Lancer le noyau CUDA
    matrixMultiplyShared<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    // Copier le résultat depuis le GPU vers la mémoire hôte
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Afficher un exemple de résultat pour validation
    std::cout << "C[0][0] (Shared): " << h_C[0] << std::endl;

    // Libération de mémoire sur l'hôte et le GPU
    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}

Overwriting matrice_multiplication_shared.cu


In [ ]:
!nvcc matrice_multiplication_shared.cu -o matrix_multiplication_shared

In [ ]:
!./matrix_multiplication_shared

C[0][0] (Shared): 512


In [ ]:
!nvprof ./matrix_multiplication_shared

==1428== NVPROF is profiling process 1428, command: ./matrix_multiplication_shared
C[0][0] (Shared): 512
==1428== Profiling application: ./matrix_multiplication_shared
==1428== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   73.31%  742.00us         1  742.00us  742.00us  742.00us  matrixMultiplyShared(float const *, float const *, float*, int)
                   17.28%  174.88us         2  87.437us  87.262us  87.613us  [CUDA memcpy HtoD]
                    9.41%  95.294us         1  95.294us  95.294us  95.294us  [CUDA memcpy DtoH]
      API calls:   96.35%  72.931ms         3  24.310ms  3.7850us  72.849ms  cudaMalloc
                    1.87%  1.4174ms         3  472.48us  252.28us  844.49us  cudaMemcpy
                    0.98%  745.13us         1  745.13us  745.13us  745.13us  cudaDeviceSynchronize
                    0.30%  228.87us         3  76.291us  13.476us  119.75us  cudaFree
                    0.28%  21

# 4. Optimiser les schémas d'accès à la mémoire partagée

In [ ]:
%%writefile matrice_multiplication_shared_optimized.cu

Writing matrice_multiplication_shared_optimized.cu


In [ ]:
%%writefile matrice_multiplication_shared_optimized.cu
#include <cuda_runtime.h>
#include <iostream>

#define TILE_SIZE 16  // Taille de la tuile (TILE_SIZE x TILE_SIZE)
#define PADDING 1         // Padding pour éviter les conflits de banque
#define N 512         // Taille des matrices carrées (N x N)

// Noyau CUDA pour la multiplication matricielle optimisée avec mémoire partagée et padding
__global__ void matrixMultiplySharedOptimized(const float* A, const float* B, float* C, int n) {
    // Déclaration de mémoire partagée avec padding
    __shared__ float tile_A[TILE_SIZE][TILE_SIZE + PADDING];
    __shared__ float tile_B[TILE_SIZE][TILE_SIZE + PADDING];

    // Calculer les indices de ligne et de colonne pour le thread actuel
    int row = blockIdx.y * TILE_SIZE + threadIdx.y; // Ligne gérée par ce thread
    int col = blockIdx.x * TILE_SIZE + threadIdx.x; // Colonne gérée par ce thread
    float sum = 0.0f; // Accumulateur pour le produit-somme

    // Boucle sur toutes les tuiles nécessaires
    for (int tileIdx = 0; tileIdx < (n + TILE_SIZE - 1) / TILE_SIZE; ++tileIdx) {
        // Charger une tuile de A dans la mémoire partagée avec padding
        if (row < n && (tileIdx * TILE_SIZE + threadIdx.x) < n)
            tile_A[threadIdx.y][threadIdx.x] = A[row * n + tileIdx * TILE_SIZE + threadIdx.x];
        else
            tile_A[threadIdx.y][threadIdx.x] = 0.0f;

        // Charger une tuile de B dans la mémoire partagée avec padding
        if (col < n && (tileIdx * TILE_SIZE + threadIdx.y) < n)
            tile_B[threadIdx.y][threadIdx.x] = B[(tileIdx * TILE_SIZE + threadIdx.y) * n + col];
        else
            tile_B[threadIdx.y][threadIdx.x] = 0.0f;

        // Synchroniser les threads pour s'assurer que la tuile est complètement chargée
        __syncthreads();

        // Calcul produit-somme sur la tuile actuelle
        for (int k = 0; k < TILE_SIZE; ++k) {
            sum += tile_A[threadIdx.y][k] * tile_B[k][threadIdx.x];
        }

        // Synchroniser les threads avant de charger la prochaine tuile
        __syncthreads();
    }

    // Stocker le résultat final dans la mémoire globale
    if (row < n && col < n) {
        C[row * n + col] = sum;
    }
}

int main() {
    int size = N * N * sizeof(float); // Taille totale en octets des matrices
    float *h_A, *h_B, *h_C;          // Pointeurs pour la mémoire hôte
    float *d_A, *d_B, *d_C;          // Pointeurs pour la mémoire périphérique (GPU)

    // Allocation de mémoire sur l'hôte
    h_A = (float*)malloc(size);
    h_B = (float*)malloc(size);
    h_C = (float*)malloc(size);

    // Initialisation des matrices A et B
    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f; // Chaque élément de A est 1.0
        h_B[i] = 1.0f; // Chaque élément de B est 1.0
    }

    // Allocation de mémoire sur le GPU
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copier les matrices A et B depuis l'hôte vers le GPU
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Configuration des dimensions des threads et des blocs
    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE); // Chaque bloc contient TILE_SIZE x TILE_SIZE threads
    dim3 blocksPerGrid((N + TILE_SIZE - 1) / TILE_SIZE,
                       (N + TILE_SIZE - 1) / TILE_SIZE); // Nombre de blocs nécessaires

    // Lancer le noyau CUDA optimisé
    matrixMultiplySharedOptimized<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    // Copier le résultat depuis le GPU vers la mémoire hôte
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Afficher un exemple de résultat pour validation
    std::cout << "C[0][0] (Shared Optimized): " << h_C[0] << std::endl;

    // Libération de mémoire sur l'hôte et le GPU
    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Overwriting matrice_multiplication_shared_optimized.cu


In [ ]:
!nvcc matrice_multiplication_shared_optimized.cu -o matrix_multiplication_shared_optimized

In [ ]:
!./matrix_multiplication_shared_optimized

C[0][0] (Shared Optimized): 512


In [ ]:
!nvprof ./matrix_multiplication_shared_optimized

==1621== NVPROF is profiling process 1621, command: ./matrix_multiplication_shared_optimized
C[0][0] (Shared Optimized): 512
==1621== Profiling application: ./matrix_multiplication_shared_optimized
==1621== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   78.45%  975.69us         1  975.69us  975.69us  975.69us  matrixMultiplySharedOptimized(float const *, float const *, float*, int)
                   14.04%  174.65us         2  87.326us  87.294us  87.358us  [CUDA memcpy HtoD]
                    7.51%  93.438us         1  93.438us  93.438us  93.438us  [CUDA memcpy DtoH]
      API calls:   96.65%  90.084ms         3  30.028ms  6.2770us  90.007ms  cudaMalloc
                    1.63%  1.5182ms         3  506.07us  254.94us  948.80us  cudaMemcpy
                    1.05%  978.32us         1  978.32us  978.32us  978.32us  cudaDeviceSynchronize
                    0.26%  240.11us         3  80.037us  14.679us  124.65us 

# 5. Profilage des performances

In [ ]:
%%writefile matrice_multiplication_profiling.cu

Writing matrice_multiplication_profiling.cu


In [ ]:
%%writefile matrice_multiplication_profiling.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>

// Noyau pour la multiplication matricielle utilisant la mémoire globale
__global__ void matMulGlobalMemory(float *A, float *B, float *C, int N) {
    // Calcul des indices de ligne et de colonne pour chaque thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Vérifier si les indices sont dans les limites de la matrice
    if (row < N && col < N) {
        float value = 0.0f; // Variable pour accumuler le produit-somme
        // Calculer la multiplication matricielle pour cet élément
        for (int k = 0; k < N; ++k) {
            value += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = value; // Stocker le résultat dans C
    }
}

int main() {
    int N = 1024; // Dimension de la matrice (N x N)
    // Allouer des matrices A, B, et C sur l'hôte (CPU)
    std::vector<float> h_A(N * N, 1.0f); // Matrice A (initialisée à 1)
    std::vector<float> h_B(N * N, 1.0f); // Matrice B (initialisée à 1)
    std::vector<float> h_C(N * N, 0.0f); // Matrice C (initialisée à 0)

    float *d_A, *d_B, *d_C;
    // Allocation mémoire sur le GPU pour A, B et C
    cudaMalloc(&d_A, N * N * sizeof(float));
    cudaMalloc(&d_B, N * N * sizeof(float));
    cudaMalloc(&d_C, N * N * sizeof(float));

    // Copier les matrices A et B depuis l'hôte vers le GPU
    cudaMemcpy(d_A, h_A.data(), N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B.data(), N * N * sizeof(float), cudaMemcpyHostToDevice);

    // Définir les dimensions des blocs et de la grille
    dim3 dimBlock(16, 16, 1);  // Taille du bloc de threads
    dim3 dimGrid((N + 15) / 16, (N + 15) / 16, 1); // Nombre de blocs nécessaires

    // Lancer le noyau pour effectuer la multiplication matricielle
    matMulGlobalMemory<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize(); // Synchroniser l'exécution du noyau

    // Copier le résultat de la matrice C du GPU vers l'hôte
    cudaMemcpy(h_C.data(), d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);

    // Afficher un exemple de résultat pour vérifier
    std::cout << "Exemple de sortie (C[0][0]): " << h_C[0] << std::endl;

    // Libérer la mémoire allouée sur le GPU
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Overwriting matrice_multiplication_profiling.cu


In [ ]:
!nvcc matrice_multiplication_profiling.cu -o matrice_multiplication_profiling

In [ ]:
!./matrice_multiplication_profiling

Exemple de sortie (C[0][0]): 1024


In [ ]:
!nvprof ./matrice_multiplication_profiling

==1804== NVPROF is profiling process 1804, command: ./matrice_multiplication_profiling
Exemple de sortie (C[0][0]): 1024
==1804== Profiling application: ./matrice_multiplication_profiling
==1804== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   80.66%  9.1159ms         1  9.1159ms  9.1159ms  9.1159ms  matMulGlobalMemory(float*, float*, float*, int)
                   13.68%  1.5458ms         2  772.88us  760.17us  785.58us  [CUDA memcpy HtoD]
                    5.66%  639.67us         1  639.67us  639.67us  639.67us  [CUDA memcpy DtoH]
      API calls:   86.55%  84.065ms         3  28.022ms  129.78us  83.787ms  cudaMalloc
                    9.39%  9.1212ms         1  9.1212ms  9.1212ms  9.1212ms  cudaDeviceSynchronize
                    3.07%  2.9787ms         3  992.90us  967.81us  1.0330ms  cudaMemcpy
                    0.55%  535.58us         3  178.53us  117.49us  210.81us  cudaFree
                    0.27%

# 6. Optimisation itérative



In [ ]:
%%writefile matrice_multiplication_iterative_optimization.cu

Writing matrice_multiplication_iterative_optimization.cu


In [ ]:
%%writefile matrice_multiplication_iterative_optimization.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>

#define TILE_WIDTH 16 // Taille de la tuile pour la multiplication optimisée

// Noyau CUDA pour la multiplication matricielle utilisant la mémoire partagée
__global__ void matMulSharedMemory(float *A, float *B, float *C, int N) {
    // Déclaration de la mémoire partagée pour les tuiles de A et B
    __shared__ float tileA[TILE_WIDTH][TILE_WIDTH];
    __shared__ float tileB[TILE_WIDTH][TILE_WIDTH];

    // Calcul des indices de ligne et de colonne pour chaque thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float value = 0.0f; // Variable pour accumuler le produit-somme

    // Boucle sur les tuiles nécessaires pour la multiplication
    for (int t = 0; t < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++t) {
        // Charger une tuile de la matrice A dans la mémoire partagée
        if (row < N && t * TILE_WIDTH + threadIdx.x < N)
            tileA[threadIdx.y][threadIdx.x] = A[row * N + t * TILE_WIDTH + threadIdx.x];
        else
            tileA[threadIdx.y][threadIdx.x] = 0.0f;

        // Charger une tuile de la matrice B dans la mémoire partagée
        if (col < N && t * TILE_WIDTH + threadIdx.y < N)
            tileB[threadIdx.y][threadIdx.x] = B[(t * TILE_WIDTH + threadIdx.y) * N + col];
        else
            tileB[threadIdx.y][threadIdx.x] = 0.0f;

        // Synchronisation des threads avant le calcul du produit-somme
        __syncthreads();

        // LE RESTE DU CODE...

        // Calculer le produit-somme pour la tuile actuelle
        for (int k = 0; k < TILE_WIDTH; ++k) {
            value += tileA[threadIdx.y][k] * tileB[k][threadIdx.x];
        }

        // Synchroniser les threads avant de charger la prochaine tuile
        __syncthreads();
    }

    // Stocker le résultat final dans la matrice C
    if (row < N && col < N) {
        C[row * N + col] = value;
    }
}

int main() {
    int N = 1024; // Dimension de la matrice (N x N)
    // Allouer des matrices A, B, et C sur l'hôte (CPU)
    std::vector<float> h_A(N * N, 1.0f); // Matrice A (initialisée à 1)
    std::vector<float> h_B(N * N, 1.0f); // Matrice B (initialisée à 1)
    std::vector<float> h_C(N * N, 0.0f); // Matrice C (initialisée à 0)

    float *d_A, *d_B, *d_C;
    // Allocation mémoire sur le GPU pour A, B et C
    cudaMalloc(&d_A, N * N * sizeof(float));
    cudaMalloc(&d_B, N * N * sizeof(float));
    cudaMalloc(&d_C, N * N * sizeof(float));

    // Copier les matrices A et B depuis l'hôte vers le GPU
    cudaMemcpy(d_A, h_A.data(), N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B.data(), N * N * sizeof(float), cudaMemcpyHostToDevice);

    // Définir les dimensions des blocs et de la grille
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH, 1);  // Taille du bloc de threads
    dim3 dimGrid((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) / TILE_WIDTH, 1); // Nombre de blocs nécessaires

    // Lancer le noyau pour effectuer la multiplication matricielle optimisée
    matMulSharedMemory<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize(); // Synchroniser l'exécution du noyau

    // Copier le résultat de la matrice C du GPU vers l'hôte
    cudaMemcpy(h_C.data(), d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);

    // Afficher un exemple de résultat pour vérifier
    std::cout << "Exemple de sortie (C[0][0]): " << h_C[0] << std::endl;

    // Libérer la mémoire allouée sur le GPU
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Overwriting matrice_multiplication_iterative_optimization.cu


In [ ]:
!nvcc matrice_multiplication_iterative_optimization.cu -o matrice_multiplication_iterative_optimization

In [ ]:
!./matrice_multiplication_iterative_optimization

Exemple de sortie (C[0][0]): 1024


In [ ]:
!nvprof ./matrice_multiplication_iterative_optimization

==1981== NVPROF is profiling process 1981, command: ./matrice_multiplication_iterative_optimization
Exemple de sortie (C[0][0]): 1024
==1981== Profiling application: ./matrice_multiplication_iterative_optimization
==1981== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   73.09%  5.8026ms         1  5.8026ms  5.8026ms  5.8026ms  matMulSharedMemory(float*, float*, float*, int)
                   18.51%  1.4696ms         2  734.82us  732.30us  737.33us  [CUDA memcpy HtoD]
                    8.40%  667.09us         1  667.09us  667.09us  667.09us  [CUDA memcpy DtoH]
      API calls:   89.99%  87.754ms         3  29.251ms  73.359us  87.602ms  cudaMalloc
                    5.95%  5.8050ms         1  5.8050ms  5.8050ms  5.8050ms  cudaDeviceSynchronize
                    3.03%  2.9536ms         3  984.54us  916.37us  1.0683ms  cudaMemcpy
                    0.56%  542.35us         3  180.78us  126.62us  208.66us  cudaFree